In [11]:
'''
run 2
'''


import numpy as np
from scipy.stats import qmc
import random
import pandas as pd


D = 4 # Dimension 
N = 100 # Number of experimental recipes


"""
Concentration of each solution, mol/L
Gd(ClO4)3 = 1
Ni_NMIDA = 0.5
NaOH = 1
UREA = 1
"""


# Volume interval for each solution
v_Gd_range = [0.5, 1.2]
v_Ni_NMIDA_range = [0.7, 1.4]
v_NaOH_range = [2., 2.5]
v_UREA_range = [0., 1.] 
v_total = 8.


# LHS sampling
sampler = qmc.LatinHypercube(d=D)
portion = sampler.random(n=N)
origin_result = qmc.scale(portion, [v_Gd_range[0], v_Ni_NMIDA_range[0], v_NaOH_range[0], v_UREA_range[0]], 
                          [v_Gd_range[1], v_Ni_NMIDA_range[1], v_NaOH_range[1], v_UREA_range[1]])


# Calculating the amount of water
v_H2O_array = np.empty([N,1])
for i in range(N):
    v_H2O_array[i] = v_total-origin_result[i][0]-origin_result[i][1]-origin_result[i][2]-origin_result[i][3]


# Saving file
v_H2O_array = v_H2O_array.T
origin_result = np.insert(origin_result,0,v_H2O_array,axis =1)
origin_result = np.round(origin_result, decimals=2, out=None)
origin_result = pd.DataFrame(origin_result,columns = ["H2O","Gd(ClO4)3","Ni-NMIDA","NaOH","UREA"])
origin_result.to_csv("run2.csv")

In [ ]:
'''
run 2 ------ 3
'''


import pandas as pd
import numpy as np
import os
import math
import copy
import random
from scipy.stats import entropy
from lazypredict.Supervised import LazyClassifier, LazyRegressor
from sklearn.model_selection import train_test_split


# Reading datas
path = "experiment_results"
files = os.listdir(path)
X_datas = []
Y_datas = []
for file in files:
    full_path = os.path.join(path, file)
    X_datas.append(pd.read_excel(full_path, usecols=["Gd(ClO4)3", "Ni-NMIDA", "NaOH", "UREA"]))
    Y_datas.append(pd.read_excel(full_path, usecols=["Crystal"]))
if len(X_datas) == 1:
    X = X_datas[0]
    Y = Y_datas[0]
else:
    X = pd.concat([X_datas[0], X_datas[1]])
    Y = pd.concat([Y_datas[0], Y_datas[1]])
    for i in range(2,len(X_datas)):
        X = pd.concat([X, X_datas[i]])
        Y = pd.concat([Y, Y_datas[i]])
X = np.array(X)
Y = np.array(Y, dtype = bool)
Y = Y.T[0]


# Training model
model_behavior = [] # list to store the model perforamnce
clfs = [] # list to store the actual model
for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.3, random_state = i, stratify=Y)
    clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
    models, predictions = clf.fit(X_train, X_test, y_train, y_test)
    models = models.drop(columns = ["Accuracy", "ROC AUC", "F1 Score", "Time Taken"])
    model_behavior.append(models)
    clfs.append(clf)


# Laying out points
all_points = []
v_Gd_p = np.linspace(0.5, 1.2, 71)
v_Ni_NMIDA_p = np.linspace(0.7, 1.4, 71)
v_NaOH_p = np.linspace(2., 2.5, 51)
v_UREA_p = np.linspace(0., 1., 101)
for i in v_Gd_p:
    for j in v_Ni_NMIDA_p:
        for p in v_NaOH_p:
            for q in v_UREA_p:
                all_points.append([i, j, p, q])
all_points = np.array(all_points)


# Predicting entropy
all_probability = []
for i in range(100):
    try:
        # the probability to predict all the candidates from different models, but some model can not do predict_proba
        result_temp = clf.models[model_behavior[i].index[0]].predict_proba(all_points)
        all_probability.append(result_temp)
    except:
        pass
all_probability = np.array(all_probability)
# calculate the mean from different models
ensemble_probability = all_probability.mean(axis = 0)
point_entropy = entropy(ensemble_probability, axis = 1)
sorted_points = all_points[np.argsort(point_entropy)[::-1]]


# Getting new recipes
all_p = [sorted_points[0]]
while len(all_p)<20:
    for candidate in sorted_points:
        if np.linalg.norm(np.array(all_p) - candidate, axis = 1).min()<0.5: # if the total volume diff is smaller than 0.5 mL, we ignore the point as they are too small
            pass 
        else:
            all_p.append(candidate)
            break


# Saving file
v_total = 8.
v_H2O_array = np.empty([20,1])
for i in range(20):
    v_H2O_array[i] = v_total-all_p[i][0]-all_p[i][1]-all_p[i][2]-all_p[i][3]
v_H2O_array = v_H2O_array.T
all_p = np.insert(all_p, 0, v_H2O_array, axis =1)
all_p = np.round(all_p, decimals=2, out=None)
all_p = pd.DataFrame(all_p,columns = ["H2O","Gd(ClO4)3","Ni-NMIDA","NaOH","UREA"])
all_p.to_csv("run3.csv") # or all_p.to_csv("run4.csv")

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:00<00:00, 32.11it/s]


[LightGBM] [Info] Number of positive: 109, number of negative: 311
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000054 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 345
[LightGBM] [Info] Number of data points in the train set: 420, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.259524 -> initscore=-1.048445
[LightGBM] [Info] Start training from score -1.048445
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:00<00:00, 35.82it/s]


[LightGBM] [Info] Number of positive: 109, number of negative: 311
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000116 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 339
[LightGBM] [Info] Number of data points in the train set: 420, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.259524 -> initscore=-1.048445
[LightGBM] [Info] Start training from score -1.048445
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:00<00:00, 33.48it/s]


...

[LightGBM] [Info] Number of positive: 109, number of negative: 311
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000047 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 339
[LightGBM] [Info] Number of data points in the train set: 420, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.259524 -> initscore=-1.048445
[LightGBM] [Info] Start training from score -1.048445
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:00<00:00, 32.35it/s]


[LightGBM] [Info] Number of positive: 109, number of negative: 311
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000046 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 344
[LightGBM] [Info] Number of data points in the train set: 420, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.259524 -> initscore=-1.048445
[LightGBM] [Info] Start training from score -1.048445
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:00<00:00, 34.01it/s]


[LightGBM] [Info] Number of positive: 109, number of negative: 311
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000044 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 341
[LightGBM] [Info] Number of data points in the train set: 420, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.259524 -> initscore=-1.048445
[LightGBM] [Info] Start training from score -1.048445
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

In [2]:
'''
run 5
'''


import numpy as np
from scipy.stats import qmc
import random
import pandas as pd


D = 4 # Dimension 
N = 100 # Number of experimental recipes


"""
Concentration of each solution, mol/L
Gd(ClO4)3 = 1
Ni_NMIDA = 0.5
NaOH = 1
UREA = 1
"""


# Volume interval for each solution
v_Gd_range = [0.9, 1.2]
v_Ni_NMIDA_range = [1.2, 2.]
v_NaOH_range = [1.5, 2.5]
v_UREA_range = [0., 1.] 
v_total = 8.


# LHS sampling
sampler = qmc.LatinHypercube(d=D)
portion = sampler.random(n=N)
origin_result = qmc.scale(portion, [v_Gd_range[0], v_Ni_NMIDA_range[0], v_NaOH_range[0], v_UREA_range[0]], 
                          [v_Gd_range[1], v_Ni_NMIDA_range[1], v_NaOH_range[1], v_UREA_range[1]])


# Calculating the amount of water
v_H2O_array = np.empty([N,1])
for i in range(N):
    v_H2O_array[i] = v_total-origin_result[i][0]-origin_result[i][1]-origin_result[i][2]-origin_result[i][3]


# Saving file   
v_H2O_array = v_H2O_array.T
origin_result = np.insert(origin_result,0,v_H2O_array,axis =1)
origin_result = np.round(origin_result, decimals=2, out=None)
origin_result = pd.DataFrame(origin_result,columns = ["H2O","Gd(ClO4)3","Ni-NMIDA","NaOH","UREA"])
origin_result.to_csv("run5.csv")

In [ ]:
'''
run 6 ------ 7
'''


import pandas as pd
import numpy as np
import os
import math
import copy
import random
from scipy.stats import entropy
from lazypredict.Supervised import LazyClassifier, LazyRegressor
from sklearn.model_selection import train_test_split


# Reading datas
path = "experiment_results"
files = os.listdir(path)
X_datas = []
Y_datas = []
for file in files:
    full_path = os.path.join(path, file)
    X_datas.append(pd.read_excel(full_path, usecols=["Gd(ClO4)3", "Ni-NMIDA", "NaOH", "UREA"]))
    Y_datas.append(pd.read_excel(full_path, usecols=["Crystal"]))
if len(X_datas) == 1:
    X = X_datas[0]
    Y = Y_datas[0]
else:
    X = pd.concat([X_datas[0], X_datas[1]])
    Y = pd.concat([Y_datas[0], Y_datas[1]])
    for i in range(2,len(X_datas)):
        X = pd.concat([X, X_datas[i]])
        Y = pd.concat([Y, Y_datas[i]])
X = np.array(X)
Y = np.array(Y, dtype = bool)
Y = Y.T[0]


# Training model
model_behavior = [] # list to store the model perforamnce
clfs = [] # list to store the actual model
for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.3, random_state = i, stratify=Y)
    clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
    models, predictions = clf.fit(X_train, X_test, y_train, y_test)
    models = models.drop(columns = ["Accuracy", "ROC AUC", "F1 Score", "Time Taken"])
    model_behavior.append(models)
    clfs.append(clf)


# Laying out points
all_points = []
v_Gd_p = np.linspace(0.5, 1.2, 71)
v_Ni_NMIDA_p = np.linspace(0.7, 2., 131)
v_NaOH_p = np.linspace(0.5, 2.5, 201)
v_UREA_p = np.linspace(0., 1., 101)
for i in v_Gd_p:
    for j in v_Ni_NMIDA_p:
        for p in v_NaOH_p:
            for q in v_UREA_p:
                all_points.append([i, j, p, q])
all_points = np.array(all_points)


# Predicting entropy
all_probability = []
for i in range(100):
    try:
        # the probability to predict all the candidates from different models, but some model can not do predict_proba
        result_temp = clf.models[model_behavior[i].index[0]].predict_proba(all_points)
        all_probability.append(result_temp)
    except:
        pass
all_probability = np.array(all_probability)
# calculate the mean from different models
ensemble_probability = all_probability.mean(axis = 0)
point_entropy = entropy(ensemble_probability, axis = 1)
sorted_points = all_points[np.argsort(point_entropy)[::-1]]


# Getting new recipes
all_p = [sorted_points[0]]
while len(all_p)<20:
    for candidate in sorted_points:
        if np.linalg.norm(np.array(all_p) - candidate, axis = 1).min()<0.5: # if the total volume diff is smaller than 0.5 mL, we ignore the point as they are too small
            pass 
        else:
            all_p.append(candidate)
            break


# Saving file
v_total = 8.
v_H2O_array = np.empty([20,1])
for i in range(20):
    v_H2O_array[i] = v_total-all_p[i][0]-all_p[i][1]-all_p[i][2]-all_p[i][3]
v_H2O_array = v_H2O_array.T
all_p = np.insert(all_p, 0, v_H2O_array, axis =1)
all_p = np.round(all_p, decimals=2, out=None)
all_p = pd.DataFrame(all_p,columns = ["H2O","Gd(ClO4)3","Ni-NMIDA","NaOH","UREA"])
all_p.to_csv("run6.csv") # or all_p.to_csv("run7.csv")